In [1]:
from dotenv import load_dotenv
from agents import Runner, Agent, trace
from agents.mcp import MCPServerStdio
from IPython.display import display, Markdown
import os
from datetime import datetime

In [2]:
load_dotenv(override=True)

True

In [3]:
params = {"command": "npx","args": ["-y", "mcp-memory-libsql"],"env": {"LIBSQL_URL": "file:./memory/vimal.db"}}
async with MCPServerStdio(params=params,client_session_timeout_seconds=60) as server:
    mcp_tools = await server.list_tools()

In [4]:
mcp_tools

[Tool(name='create_entities', title=None, description='Create new entities with observations and optional embeddings', inputSchema={'type': 'object', 'properties': {'entities': {'type': 'array', 'items': {'type': 'object', 'properties': {'name': {'type': 'string'}, 'entityType': {'type': 'string'}, 'observations': {'type': 'array', 'items': {'type': 'string'}}, 'embedding': {'type': 'array', 'items': {'type': 'number'}, 'description': 'Optional vector embedding for similarity search'}}, 'required': ['name', 'entityType', 'observations']}}}, 'required': ['entities']}, outputSchema=None, annotations=None, meta=None),
 Tool(name='search_nodes', title=None, description='Search for entities and their relations using text or vector similarity', inputSchema={'type': 'object', 'properties': {'query': {'oneOf': [{'type': 'string', 'description': 'Text search query'}, {'type': 'array', 'items': {'type': 'number'}, 'description': 'Vector for similarity search'}]}}, 'required': ['query']}, outputS

In [5]:
instructions = "You use your entity tools as a persistent memory to store and recall information about your conversations. Store every conversation."
request = "My name's Vimal. I'm an ML engineer. I am trying out MCP."
model = "gpt-4.1"

In [6]:
async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as mcp_server:
    agent = Agent(name="agent", instructions=instructions, model=model,mcp_servers=[mcp_server])
    with trace("conversation"):
        result = await Runner.run(agent, request)
    display(Markdown(result.final_output))

Nice to meet you, Vimal! I've noted that you're an ML engineer and are currently trying out MCP. If you want to explore or discuss anything related to ML, MCP, or anything else, just let me know!

In [7]:
async with MCPServerStdio(params=params,client_session_timeout_seconds=60) as mcp_server:
    agent = Agent(name="agent", instructions=instructions, model=model, mcp_servers=[mcp_server])
    with trace("conversation"):
        result = await Runner.run(agent, "My name is Vimal. What do you know about me?")
    display(Markdown(result.final_output))

Here’s what I know about you so far:

- Your name is Vimal.
- You are an ML (Machine Learning) engineer.
- You are trying out MCP.

If there’s anything else you’d like me to remember or update, just let me know!

In [13]:
playwright_params = {"command": "npx","args": ["-y", "@playwright/mcp@latest"]}

instructions = """
You browse the internet to accomplish your instructions.
You are highly capable at browsing the internet independently to accomplish your task, 
including accepting all cookies and clicking 'not now' as
appropriate to get to the content you need. If one website isn't fruitful, try another. 
Be persistent until you have solved your assignment,
trying different options and sites as needed.
"""

request = f"Please research the latest news on Amazon stock price and briefly summarize its outlook. \
For context, the current date is {datetime.now().strftime('%Y-%m-%d')}"

model = "gpt-4.1-mini"


In [14]:
async with MCPServerStdio(params=playwright_params, client_session_timeout_seconds=60) as mcp_browser_server:
    agent = Agent(name="agent", instructions=instructions, model=model, mcp_servers=[mcp_browser_server])
    with trace("conversation"):
        result = await Runner.run(agent, request)
    display(Markdown(result.final_output))

The latest outlook on Amazon's stock, as of August 2025, is positive due to its position as a tactical winner in the event of a potential back-half rate cut. According to Ken Gawrelski, a senior internet analyst at Wells Fargo, Amazon is strategically positioned to benefit if interest rates are cut in the latter part of the year. This analyst insight indicates favorable prospects for Amazon stock, supported by broader market expectations of easing monetary policy which could enhance growth opportunities for large tech companies like Amazon. The stock price recently closed at $228.84, showing a 3.10% increase. 

In summary, the outlook for Amazon stock is optimistic, especially with the anticipation of rate cuts that could drive further gains in the second half of 2025.

## USING POLYGON API WRAPPED IN MCP SERVER

In [19]:
params = {"command": "uv", "args": ["run", "market_server.py"]}
async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as server:
    mcp_tools = await server.list_tools()
mcp_tools

[Tool(name='lookup_share_price', title=None, description='This tool provides the current price of the given stock symbol.\n\n    Args:\n        symbol: the symbol of the stock\n    ', inputSchema={'properties': {'symbol': {'title': 'Symbol', 'type': 'string'}}, 'required': ['symbol'], 'title': 'lookup_share_priceArguments', 'type': 'object'}, outputSchema={'properties': {'result': {'title': 'Result', 'type': 'number'}}, 'required': ['result'], 'title': 'lookup_share_priceOutput', 'type': 'object'}, annotations=None, meta=None)]

In [20]:
instructions = "You are an Stock Market Expert. You answer questions about the stock market."
request = "What is the share price of Nvidia?"
model = "gpt-4.1"
async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as server:
    agent = Agent(name="agent",instructions=instructions,model=model,mcp_servers=[server])
    with trace("stock_market"):
        result = await Runner.run(agent, request)
    display(Markdown(result.final_output))

The current share price of Nvidia (NVDA) is $177.99. If you need more detailed data or the latest updates, let me know!